Dependencies

In [2]:
pip install pandas numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Data Download TODO: Make this data retrieval from Dropbox

In [7]:
!wget https://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz 

--2024-12-04 23:11:31--  https://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz
Resolving web.eecs.umich.edu (web.eecs.umich.edu)... 141.212.113.214
Connecting to web.eecs.umich.edu (web.eecs.umich.edu)|141.212.113.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10878146150 (10G) [application/x-gzip]
Saving to: ‘MELD.Raw.tar.gz.1’

MELD.Raw.tar.gz.1     0%[                    ]  67.83M  17.9MB/s    eta 10m 28s^C


In [ ]:
# extract command
!tar -xzvf MELD.Raw.tar.gz

x MELD.Raw/
x MELD.Raw/train.tar.gz
x MELD.Raw/dev.tar.gz
x MELD.Raw/test_sent_emo.csv
x MELD.Raw/dev_sent_emo.csv
x MELD.Raw/README.txt
x MELD.Raw/test.tar.gz


In [1]:
# list mp4 file names
!tar -xzvf MELD.Raw/train.tar.gz
!tar -xzvf MELD.Raw/test.tar.gz
!tar -xzvf MELD.Raw/dev.tar.gz

dev_splits_complete/dia97_utt3.mp4
dev_splits_complete/dia32_utt4.mp4
dev_splits_complete/dia18_utt8.mp4
dev_splits_complete/dia30_utt18.mp4
dev_splits_complete/dia13_utt10.mp4
dev_splits_complete/dia27_utt7.mp4
dev_splits_complete/dia32_utt13.mp4
dev_splits_complete/dia4_utt14.mp4
dev_splits_complete/dia47_utt2.mp4
dev_splits_complete/dia9_utt7.mp4
dev_splits_complete/dia56_utt10.mp4
dev_splits_complete/dia10_utt5.mp4
dev_splits_complete/dia84_utt3.mp4
dev_splits_complete/dia97_utt15.mp4
dev_splits_complete/dia19_utt16.mp4
dev_splits_complete/dia30_utt0.mp4
dev_splits_complete/dia16_utt4.mp4
dev_splits_complete/dia104_utt3.mp4
dev_splits_complete/dia85_utt13.mp4
dev_splits_complete/dia25_utt3.mp4
dev_splits_complete/dia113_utt2.mp4
dev_splits_complete/dia53_utt5.mp4
dev_splits_complete/dia1_utt4.mp4
dev_splits_complete/dia7_utt3.mp4
dev_splits_complete/dia73_utt0.mp4
dev_splits_complete/dia62_utt4.mp4
dev_splits_complete/dia59_utt4.mp4
dev_splits_complete/dia90_utt6.mp4
dev_splits_com

STOP and go to process_data.py

Constants

Data Processing

First, Extract the raw text data.

In [ ]:
def csv_to_text_parsed_df(fname):
    print("fname", fname)
    df = pd.read_csv(fname)
    # List of columns to keep
    columns_to_keep = ["Emotion", "Dialogue_ID", "Utterance_ID", "Utterance"]

    # Drop columns not in columns_to_keep
    df = df.drop(
        [col for col in df.columns if col not in columns_to_keep],
        axis=1
    )
    df.groupby("Emotion").apply(lambda x: x)
    df = df.set_index(["Emotion"])
    df.sort_values(by=["Emotion", "Dialogue_ID", "Utterance_ID"], inplace=True)
    return df

train_df = csv_to_text_parsed_df("MELD.Raw/test_sent_emo.csv")
test_df = csv_to_text_parsed_df("MELD.Raw/train_sent_emo.csv")

fname MELD.Raw/test_sent_emo.csv
fname MELD.Raw/train_sent_emo.csv


/var/folders/df/v_9sm4n523gdywb561ld9c6w0000gn/T/ipykernel_7882/3463960993.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("Emotion").apply(lambda x: x)
/var/folders/df/v_9sm4n523gdywb561ld9c6w0000gn/T/ipykernel_7882/3463960993.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("Emotion").apply(lambda x: x)


In [6]:
train_df
test_df

,Utterance,Dialogue_ID,Utterance_ID
Emotion,,,
anger,"Oh no-no-no, give me some specifics.",6,5
anger,You fell asleep!!,6,9
anger,There was no kangaroo!,6,10
anger,They didnt take any of my suggestions!,6,11
anger,This guy fell asleep!,6,18
...,...,...,...
surprise,You're kidding right?,1036,4
surprise,What?!,1036,6
surprise,"Dont tell me, because of the big nostril thing?",1038,4


Next, parse and match mp4 files

In [7]:
import re
import os

pattern = r"dia(\d+)_utt(\d+)" 

def match_mp4_binary_to_df(df, mp4_files):
    df["MP4_fname"] = None
    for file in os.listdir(mp4_files):
        match = re.search(pattern, file)
        if match:
            dialogue_id = int(match.group(1))
            utterance_id = int(match.group(2))
            
            # locate the matching row in the DataFrame
            row_index = df[(df["Dialogue_ID"] == dialogue_id) & (df["Utterance_ID"] == utterance_id)].index
            if len(row_index) > 0:
                df.loc[row_index, "MP4_fname"] = file
    df = df.drop(["Dialogue_ID", "Utterance_ID"], axis=1, inplace=True)

match_mp4_binary_to_df(train_df, "train_splits")
match_mp4_binary_to_df(test_df, "output_repeated_splits_test")
train_df
test_df

,Utterance,MP4_fname
Emotion,,
anger,"Oh no-no-no, give me some specifics.",dia145_utt6.mp4
anger,You fell asleep!!,dia145_utt6.mp4
anger,There was no kangaroo!,dia145_utt6.mp4
anger,They didnt take any of my suggestions!,dia145_utt6.mp4
anger,This guy fell asleep!,dia145_utt6.mp4
...,...,...
surprise,You're kidding right?,dia232_utt9.mp4
surprise,What?!,dia232_utt9.mp4
surprise,"Dont tell me, because of the big nostril thing?",dia232_utt9.mp4
